# **СЗП**-бот
![alt text](free-png.ru-22-1.png)

In [9]:
# BigTestingDot @KinickTestingBot (https://t.me/KinickTestingBot)

import pandas as pd
import numpy as np
import os
import sqlite3
import telebot
from telebot import types
from requests.exceptions import ConnectionError

# import time
# import datetime
from datetime import datetime

# Токен
TOKEN = "7228937917:AAEhRzMDr7iC0MXwmF6vJeZ8UhOt5iKwnVM"  # token BigTestingDot
# TOKEN = '7379258385:AAGHVrQrA5taZB0WJ4nGbFyotFS1w6BxCMY' # token Олега

# пути
dir_path = os.getcwd().replace("\\", "/") + "/"
db_path = dir_path + "db.sqlite"
data_path = dir_path + "data/"

# Создание экземпляра бота
bot = telebot.TeleBot(TOKEN)

messages = {}
reg_name_eng = {}
period = {}
level = {}

rus_dict = {
    "Динамика достижения ЦУ по СЗП": "_1.png",
    "Тепловая карта – врачи": "_2.png",
    "Тепловая карта – СМП": "_3.png",
    "ЦУ по СЗП не достигнут": "_4.png",
    "ССЧ и ФОТ по категориям": "_5.png",
    "Доля ССЧ АУП": "_6.png",
}

sub_dict = {
    "Динамика достижения ЦУ по СЗП": "_1.png",
    "ЦУ по СЗП не достигнут": "_2.png",
    "ТОП МО с высоким ЦУ": "_3.png",
    "ТОП МО с низким ЦУ": "_4.png",
    "ССЧ и ФОТ по категориям": "_5.png",
    "Доля ССЧ АУП": "_6.png",
}


# меню (меню)
@bot.message_handler(commands=["start"])
def start(message):
    bot.delete_message(message.chat.id, message.message_id)
    bot.send_message(
        message.chat.id,
        text=f"""
        *Здравствуйте, {message.from_user.first_name}!*
    Этот чат-бот позволяет получить данные...""",
        parse_mode="Markdown",
    )
    bot.send_message(
        message.chat.id,
        text=f"""Для выбора режима работы воспользуйтесть меню в левом нижнем углу📲
Для смены отчётного периода (год/квартал) можно указать необходимый период в строке сообщения, не используя меню""",
        parse_mode="Markdown",
    )


# вся Россия (меню)
@bot.message_handler(commands=["all_states"])
def all(message):
    user_name = message.from_user.first_name
    user_id = message.from_user.id
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=3)
    btn1 = types.KeyboardButton("год")
    btn2 = types.KeyboardButton("квартал")
    markup.add(btn1, btn2)

    bot.send_message(
        message.chat.id,
        f"*{user_name}, выберете период*",
        reply_markup=markup,
        parse_mode="Markdown",
    )
    bot.delete_message(message.chat.id, message.message_id)
    level[user_id] = "all"


# субъекты России (меню)
@bot.message_handler(commands=["sub_states"])
def sub(message):
    user_name = message.from_user.first_name
    user_id = message.from_user.id
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=3)
    btn1 = types.KeyboardButton("год")
    btn2 = types.KeyboardButton("квартал")
    markup.add(btn1, btn2)

    bot.send_message(
        message.chat.id,
        f"*{user_name}, выберете период*",
        parse_mode="Markdown",
        reply_markup=markup,
    )
    bot.delete_message(message.chat.id, message.message_id)
    level[user_id] = "sub"


# обработка сообщений (не меню)
@bot.message_handler(content_types=["text"])
def handle_message(message):
    global db_path, data_path, messages, reg_name_eng, df_valid_regions, period, level

    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query = "SELECT rr.reg_name_rus, rr.reg_name_eng FROM reg_ref rr"
    df_valid_regions = pd.DataFrame(cur.execute(query).fetchall())
    valid_regions_list_rus = df_valid_regions[0].tolist()

    user_name = message.from_user.first_name
    user_id = message.from_user.id
    messages[user_id] = message.text

    if messages[user_id] == "год" or messages[user_id] == "Год":
        period[user_id] = "год"
    elif messages[user_id] == "квартал" or messages[user_id] == "Квартал":
        period[user_id] = "квартал"

    # кнопки для РФ
    if (
        messages[user_id] == "год"
        or messages[user_id] == "квартал"
        or messages[user_id] == "Год"
        or messages[user_id] == "Квартал"
    ) and level[user_id] == "all":
        bot.delete_message(message.chat.id, message.message_id)
        bot.delete_message(message.chat.id, message.message_id - 1)
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=3)
        btn1 = types.KeyboardButton("Динамика достижения ЦУ по СЗП")
        btn2 = types.KeyboardButton("Тепловая карта – врачи")
        btn3 = types.KeyboardButton("Тепловая карта – СМП")
        btn4 = types.KeyboardButton("ЦУ по СЗП не достигнут")
        btn5 = types.KeyboardButton("ССЧ и ФОТ по категориям")
        btn6 = types.KeyboardButton("Доля ССЧ АУП")
        markup.add(btn1, btn2, btn3, btn4, btn5, btn6)
        try:
            answer_title_path = data_path + "Russia/title.png"
            answer_title = open(answer_title_path, "rb")
            bot.send_photo(message.chat.id, answer_title)
            answer_title.close()
        except Exception as e:
            bot.delete_message(message.chat.id, message.message_id)
            bot.send_message(message.chat.id, f"Изображение 🇷🇺 не найдено, {e}")

        bot.send_message(message.chat.id, f"Выберете отчёт для РФ", reply_markup=markup)

    # выбор названия субъекта
    elif (
        messages[user_id] == "год"
        or messages[user_id] == "квартал"
        or messages[user_id] == "Год"
        or messages[user_id] == "Квартал"
    ) and level[user_id] == "sub":
        bot.delete_message(message.chat.id, message.message_id)
        bot.delete_message(message.chat.id, message.message_id - 1)
        bot.send_message(
            message.chat.id,
            f"*Укажите субъект Российской Федерации, по которому необходимо предоставить информацию:*",
            parse_mode="Markdown",
        )

    # кнопки для субъекта
    elif messages[user_id] in valid_regions_list_rus and level[user_id] == "sub":
        bot.delete_message(message.chat.id, message.message_id)
        # bot.delete_message(message.chat.id, message.message_id - 1)
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=3)
        btn1 = types.KeyboardButton("Динамика достижения ЦУ по СЗП")
        btn2 = types.KeyboardButton("ЦУ по СЗП не достигнут")
        btn3 = types.KeyboardButton("ТОП МО с высоким ЦУ")
        btn4 = types.KeyboardButton("ТОП МО с низким ЦУ")
        btn5 = types.KeyboardButton("ССЧ и ФОТ по категориям")
        btn6 = types.KeyboardButton("Доля ССЧ АУП")
        markup.add(btn1, btn2, btn3, btn4, btn5, btn6)
        reg_name_eng[user_id] = df_valid_regions[
            df_valid_regions[0] == messages[user_id]
        ].iloc[0][1]
        try:
            bot.delete_message(message.chat.id, message.message_id - 1)
            answer_title_path = data_path + reg_name_eng[user_id] + "/title.png"
            # answer_title = open(answer_title_path, "rb")
            # bot.send_photo(message.chat.id, answer_title, reply_markup=markup)
            # answer_title.close()
            bot.send_message(message.chat.id, answer_title_path, reply_markup=markup)
        except Exception as e:
            bot.delete_message(message.chat.id, message.message_id)
            bot.send_message(
                message.chat.id, f"Изображение не найдено, {e}", reply_markup=markup
            )

    # отчёт по России
    elif messages[user_id] in rus_dict and level[user_id] == "all":
        bot.delete_message(message.chat.id, message.message_id)
        bot.send_message(message.chat.id, "⏳Отчёт формируется...")
        bot.delete_message(message.chat.id, message.message_id + 1)
        try:
            answer_path = (
                data_path
                + "Russia/"
                + period[user_id]
                + "/"
                + "Russia"
                + rus_dict.get(messages[user_id])
            )
            # answer = open(answer_path, "rb")
            # bot.send_photo(message.chat.id, answer)
            # answer.close()
            bot.send_message(message.chat.id, answer_path)
        except Exception as e:
            bot.send_message(message.chat.id, "Отчёт не найден")

    # отчёт по субъектам
    elif messages[user_id] in sub_dict and level[user_id] == "sub":
        bot.delete_message(message.chat.id, message.message_id)
        bot.send_message(message.chat.id, "⏳Отчёт формируется...")
        bot.delete_message(message.chat.id, message.message_id + 1)
        try:
            answer_path = (
                data_path
                + reg_name_eng[user_id]
                + "/"
                + period[user_id]
                + "/"
                + reg_name_eng[user_id]
                + sub_dict.get(messages[user_id])
            )
            # answer = open(answer_path, "rb")
            # bot.send_photo(message.chat.id, answer)
            # answer.close()
            bot.send_message(message.chat.id, answer_path)
        except Exception as e:
            bot.send_message(message.chat.id, "Отчёт не найден")

    # неизвестное слово
    else:
        bot.send_message(
            message.chat.id,
            """❗️Название субъекта или команда меню не распознаны❗️
    ✅ если вы находитесь в разделе *субъектов Российской Федерации* - будьте внимательнее и повторите ввод названия 🗺 субъекта Российской Федерации;
    ✅ если вы находитесь в разделе *Российской Федерации в целом* - для перехода в раздел субъектов РФ воспользуйтесь меню📲""",
            parse_mode="Markdown",
        )


# запуск бота
bot.infinity_polling()

2024-07-23 17:00:24,723 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2024-07-23 17:00:24,724 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"
